<a href="https://colab.research.google.com/github/wojiushigexiaobai/Myblog/blob/main/SFT_%E7%AE%97%E5%91%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 第一步：安装依赖

In [2]:
%%capture
# 这是一个 Jupyter Notebook 的魔法命令，用于隐藏命令的输出，让笔记本界面更整洁。
# 安装 unsloth 包，unsloth 是一个用于微调大型语言模型（LLM）的工具，可以让模型运行更快、占用更少内存。
!pip install unsloth

# 卸载当前已安装的 unsloth 包（如果已安装），然后从 GitHub 的源代码安装最新版本。
# 这样可以确保我们使用的是最新功能和修复。
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# 安装 bitsandbytes 和 unsloth_zoo 包。
# bitsandbytes 是一个用于量化和优化模型的库，可以帮助减少模型占用的内存。
# unsloth_zoo 可能包含了一些预训练模型或其他工具，方便我们使用。
!pip install bitsandbytes unsloth_zoo

## 第二步：加载预训练模型

In [3]:
from unsloth import FastLanguageModel  # 导入FastLanguageModel类，用来加载和使用模型
import torch  # 导入torch工具，用于处理模型的数学运算

max_seq_length = 2048  # 设置模型处理文本的最大长度，相当于给模型设置一个“最大容量”
dtype = None  # 设置数据类型，让模型自动选择最适合的精度
load_in_4bit = True  # 使用4位量化来节省内存，就像把大箱子压缩成小箱子

# 加载预训练模型，并获取tokenizer工具
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",  # 指定要加载的模型名称
    max_seq_length=max_seq_length,  # 使用前面设置的最大长度
    dtype=dtype,  # 使用前面设置的数据类型
    load_in_4bit=load_in_4bit,  # 使用4位量化
    # token="hf_...",  # 如果需要访问授权模型，可以在这里填入密钥
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.9: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

## 第三步：微调前测试

In [4]:
prompt_style = """以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

## 指令：
你是一位精通卜卦、星象和运势预测的算命大师。
请回答以下算命问题。

### 问题：
{}

### 回答：
<think>{}</think>"""
# 定义提示风格的字符串模板，用于格式化问题

question = "1992年闰四月初九巳时生人，女，想了解健康运势"
# 定义具体的算命问题

In [5]:
FastLanguageModel.for_inference(model)
# 准备模型以进行推理

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")
# 使用 tokenizer 对格式化后的问题进行编码，并移动到 GPU

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

# 使用模型生成回答

response = tokenizer.batch_decode(outputs)
# 解码模型生成的输出为可读文本

print(response[0])
# 打印生成的回答部分

<｜begin▁of▁sentence｜>以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

## 指令：
你是一位精通卜卦、星象和运势预测的算命大师。
请回答以下算命问题。

### 问题：
1992年闰四月初九巳时生人，女，想了解健康运势

### 回答：
<think></think>
好，用户是一位1992年闰四月初九巳时出生的女性，她想了解自己的健康运势。首先，我需要根据她的出生日期计算她的生肖和元素，这样才能更准确地分析她的运势。1992年属于中国的兔年，而兔年通常与水有关，所以她的元素应该是水兔。

接下来，我要考虑她的健康方面。水兔通常身体偏寒，容易生寒或者消化不良，所以她的健康方面可能需要注意保暖和调理。同时，水元素对她的内脏尤其是肾脏有一定影响，所以健康方面要特别关注肾脏的健康，避免受寒或过度劳累。

然后，我要分析她的运势。水兔通常性格温和，待事仔细，但可能会有些拘谨。从事的工作可能与文物、文化相关，或者在教育、医疗行业有发展的机会。她的运势中，财运方面可能会有些波折，但只要经营稳健，长期来看会有不错的收获。

最后，结合她的健康和运势，我会建议她注意保暖，适当锻炼，保持心情愉悦，同时在事业上多向有经验的师傅请教，提升自己的专业能力。这样，她的健康和运势都能得到更好的发展。
</think>

根据你的出生日期1992年闰四月初九巳时，你的生肖是兔年，属于水兔。水兔的人通常体质偏寒，容易生寒或者消化不良，建议注意保暖，适当进行中医调理。健康方面，你需要关注内脏健康，特别是肾脏，避免受寒或过度劳累。

在运势方面，水兔女性通常性格温和，待事仔细，但可能有些拘谨。从事文物、文化行业或教育、医疗行业有发展机会。财运方面需要经营稳健，长期会有不错的收获。

建议你保持心情愉悦，适当锻炼，注意劳逸结合，同时在事业上多向有经验的师傅请教，提升自己的专业能力。这样健康和运势都会得到更好的发展。<｜end▁of▁sentence｜>


## 第四步：加载数据集

In [6]:
# 定义一个用于格式化提示的多行字符串模板
train_prompt_style = """以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是一位精通八字算命、紫微斗数、风水、易经卦象、塔罗牌占卜、星象、面相手相和运势预测等方面的算命大师。
请回答以下算命问题。

### 问题：
{}

#### 回答：
<思考>
{}
</思考>
{}"""

In [7]:
# 定义结束标记（EOS_TOKEN），用于指示文本的结束
EOS_TOKEN = tokenizer.eos_token  # 必须添加结束标记

# 导入数据集加载函数
from datasets import load_dataset
# 加载指定的数据集，选择中文语言和训练集的前500条记录
dataset = load_dataset("Conard/fortune-telling", "default", split = "train[0:200]", trust_remote_code=True)
# 打印数据集的列名，查看数据集中有哪些字段
print(dataset.column_names)

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

all_details.json:   0%|          | 0.00/693k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/207 [00:00<?, ? examples/s]

['Question', 'Response', 'Complex_CoT']


In [8]:
# 定义一个函数，用于格式化数据集中的每条记录
def formatting_prompts_func(examples):
    # 从数据集中提取问题、复杂思考过程和回答
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []  # 用于存储格式化后的文本
    # 遍历每个问题、思考过程和回答，进行格式化
    for input, cot, output in zip(inputs, cots, outputs):
        # 使用字符串模板插入数据，并加上结束标记
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)  # 将格式化后的文本添加到列表中
    return {
        "text": texts,  # 返回包含所有格式化文本的字典
    }
dataset = dataset.map(formatting_prompts_func, batched = True)
dataset["text"][0]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

'以下是描述任务的指令，以及提供进一步上下文的输入。\n请写出一个适当完成请求的回答。\n在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。\n\n### 指令：\n你是一位精通八字算命、紫微斗数、风水、易经卦象、塔罗牌占卜、星象、面相手相和运势预测等方面的算命大师。\n请回答以下算命问题。\n\n### 问题：\n新房装修,大门对着电梯好不好?要如何化解?\n\n#### 回答：\n<思考>\n好的，用户问的是新房装修时大门对着电梯好不好，以及如何化解。首先，我需要回忆一下风水学中关于大门和电梯的相关知识。电梯在风水中属于动气比较强的地方，因为电梯频繁开合，会带来不稳定的气流，也就是所谓的“煞气”。大门是住宅的纳气口，如果正对电梯，可能会让这些不稳定的气流直接冲进家里，影响居住者的健康和财运。\n\n接下来，我需要确认用户的具体情况。比如，大门和电梯的距离有多远？是否正对还是稍微偏一点？不过用户没有提供这些细节，所以只能给出一般性的建议。化解的方法通常有几种：屏风或玄关、门帘、五帝钱、植物、八卦镜等。需要逐一解释这些方法的原理和使用方式，同时提醒用户要根据实际情况选择，必要时咨询专业风水师。\n\n另外，还要注意语气要亲切，避免使用过于专业的术语，让用户容易理解。同时，要强调这些是传统方法，效果因人而异，保持客观中立。最后，可以建议用户如果情况复杂，最好请专业人士实地查看，这样更稳妥。\n\n</思考>\n根据传统风水学的观点，大门正对电梯易形成"开口煞"，电梯频繁升降会扰乱家宅气场。建议化解方案：\n\n1. 玄关阻隔法\n在入门处设置L型屏风或文化砖玄关墙，高度以1.8米为宜，既保持采光又形成缓冲带\n\n2. 五行通关法\n门槛石下埋设五帝钱+白玉葫芦，建议选丙申年铸造的真品古币，配合门楣悬挂九宫八卦镜\n\n3. 光影化解术\n安装磨砂玻璃内推门，门框镶嵌黄铜门槛，每日辰时用海盐净化门廊区域\n\n4. 现代科技方案\n入户区安装智能感应灯带，设置循环播放的流水声效，运用声光电技术平衡磁场\n\n需注意电梯井方位与家主命卦的关系，建议提供具体户型平面图进行吉凶方位测算。当代建筑中可采用半透明艺术隔断结合空气净化系统，既符合科学原理又兼顾传统智慧。<｜end▁of▁sentence｜>'

## 第五步：执行微调

In [9]:
FastLanguageModel.for_training(model)
model = FastLanguageModel.get_peft_model(
    model,  # 传入已经加载好的预训练模型
    r = 16,  # 设置 LoRA 的秩，决定添加的可训练参数数量
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], # 指定模型中需要微调的关键模块
    lora_alpha = 16,  # 设置 LoRA 的超参数，影响可训练参数的训练方式
    lora_dropout = 0,  # 设置防止过拟合的参数，这里设置为 0 表示不丢弃任何参数
    bias = "none",  # 设置是否添加偏置项，这里设置为 "none" 表示不添加
    use_gradient_checkpointing = "unsloth",  # 使用优化技术节省显存并支持更大的批量大小
    random_state = 3407,  # 设置随机种子，确保每次运行代码时模型的初始化方式相同
    use_rslora = False,  # 设置是否使用 Rank Stabilized LoRA 技术，这里设置为 False 表示不使用
    loftq_config = None,  # 设置是否使用 LoftQ 技术，这里设置为 None 表示不使用
)

Unsloth 2025.3.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [10]:
from trl import SFTTrainer  # 导入 SFTTrainer，用于监督式微调
from transformers import TrainingArguments  # 导入 TrainingArguments，用于设置训练参数
from unsloth import is_bfloat16_supported  # 导入函数，检查是否支持 bfloat16 数据格式

trainer = SFTTrainer(
    model=model,  # 传入要微调的模型
    tokenizer=tokenizer,  # 传入 tokenizer，用于处理文本数据
    train_dataset=dataset,  # 传入训练数据集
    dataset_text_field="text",  # 指定数据集中文本字段的名称
    max_seq_length=max_seq_length,  # 设置最大序列长度
    dataset_num_proc=2,  # 设置数据处理的并行进程数
    packing=False,  # 是否启用打包功能（这里设置为 False，打包可以让训练更快，但可能影响效果）
    args=TrainingArguments(  # 定义训练参数
        per_device_train_batch_size=2,  # 每个设备（如 GPU）上的批量大小
        gradient_accumulation_steps=4,  # 梯度累积步数，用于模拟大批次训练
        warmup_steps=5,  # 预热步数，训练开始时学习率逐渐增加的步数
        max_steps=75,  # 最大训练步数
        learning_rate=2e-4,  # 学习率，模型学习新知识的速度
        fp16=not is_bfloat16_supported(),  # 是否使用 fp16 格式加速训练（如果环境不支持 float16）
        bf16=is_bfloat16_supported(),  # 是否使用 bfloat16 格式加速训练（如果环境支持）
        logging_steps=1,  # 每隔多少步记录一次训练日志
        optim="adamw_8bit",  # 使用的优化器，用于调整模型参数
        weight_decay=0.01,  # 权重衰减，防止模型过拟合
        lr_scheduler_type="linear",  # 学习率调度器类型，控制学习率的变化方式
        seed=3407,  # 随机种子，确保训练结果可复现
        output_dir="outputs",  # 训练结果保存的目录
        report_to="none",  # 是否将训练结果报告到外部工具（如 WandB），这里设置为不报告
    ),
)

Tokenizing to ["text"] (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 200 | Num Epochs = 3 | Total steps = 75
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/4,670,623,744 (0.90% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.328200
2,2.305700
3,2.473400
4,2.182300
5,2.051500
6,2.257500
7,1.967800
8,1.972900
9,1.911800
10,1.905400


## 第七步：微调后测试

In [11]:
print(question)  # 打印前面的问题

1992年闰四月初九巳时生人，女，想了解健康运势


In [12]:
# 将模型切换到推理模式，准备回答问题
FastLanguageModel.for_inference(model)

# 将问题转换成模型能理解的格式，并发送到 GPU 上
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# 让模型根据问题生成回答，最多生成 4000 个新词
outputs = model.generate(
    input_ids=inputs.input_ids,  # 输入的数字序列
    attention_mask=inputs.attention_mask,  # 注意力遮罩，帮助模型理解哪些部分重要
    max_new_tokens=4000,  # 最多生成 4000 个新词
    use_cache=True,  # 使用缓存加速生成
)

# 将生成的回答从数字转换回文字
response = tokenizer.batch_decode(outputs)

# 打印回答
print(response[0])

<｜begin▁of▁sentence｜>以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

## 指令：
你是一位精通卜卦、星象和运势预测的算命大师。
请回答以下算命问题。

### 问题：
1992年闰四月初九巳时生人，女，想了解健康运势

### 回答：
<think></think>
好的，我现在需要帮一位1992年闰四月初九巳时出生的女性进行健康运势的预测。首先，我要了解她的生日和生辰时间，这样才能准确地分析她的八字和运势。闰四月初九巳时，属于春季，生肖是虎年，因为1992年是虎年，对应的是壬寅年。

接下来，我需要确定她的生辰八字。八字是根据出生年、月、日、时辰来计算的。1992年是壬寅年，所以她的年支是寅。闰四月属于春季，初九对应的是辰位，所以她的日支是辰。巳时对应的是火星，代表她的时支。巳是第三个时辰，属于阳火，象征热情和活力。

根据这些元素，我可以分析她的五行和性格特征。火星是主星，代表热情、活力和创造力。她的五行可能偏火，性格可能比较热情、好动。她的运势方面，火星通常与事业、创造力、爱情相关，所以她可能在事业上有创造力，或者在爱情中有热情的表现。

健康方面，火星代表活力，但过于强势的火星可能会导致压力或过度劳累。她需要注意身体的平衡，避免过度劳累，保持健康的生活习惯，比如适当的运动、良好的作息和充足的休息。

总的来说，她的健康运势看起来偏向活力和创造力，但需要注意避免过度使用体力，保持内外平衡，这样才能维持长期的健康和幸福。
</think>

1992年闰四月初九巳时出生的女性，属于壬寅年、春季，初九巳时。她的八字为寅（年支）、辰（日支）、巳（时支）。火星作为主星，代表热情、活力和创造力。

在健康方面，火星象征活力，但过度使用可能导致压力或过度劳累。建议保持健康的生活习惯，如适当运动、良好作息和充足休息，以保持内外平衡，维持长期健康和幸福。<｜end▁of▁sentence｜>


## 第八步：将微调后的模型保存为GGUF格式
GGUF格式是 用于存储LLM模型数据的高效量化格式

In [ ]:
# 导入 Google Colab 的 userdata 模块，用于访问用户数据
from google.colab import userdata

# 从 Google Colab 用户数据中获取 Hugging Face 的 API 令牌
HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN')

# 将模型保存为 8 位量化格式 (Q8_0)
# 这种格式文件小且运行快，适合部署到资源受限的设备
if True: model.save_pretrained_gguf("model", tokenizer,)

# 将模型保存为 16 位量化格式 (f16)
# 16 位量化精度更高，但文件稍大
if False: model.save_pretrained_gguf("model_f16", tokenizer, quantization_method = "f16")

# 将模型保存为 4 位量化格式 (q4_k_m)
# 4 位量化文件最小，但精度可能稍低
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

## 第九步：将微调后的模型上传到Huggingface

In [ ]:
# 导入 Hugging Face Hub 的 create_repo 函数，用于创建一个新的模型仓库
from huggingface_hub import create_repo

# 在 Hugging Face Hub 上创建一个新的模型仓库
create_repo("JanV/SFT01", token=HUGGINGFACE_TOKEN, exist_ok=True)

# 将模型和分词器上传到 Hugging Face Hub 上的仓库
model.push_to_hub_gguf("JanV/SFT01", tokenizer, token=HUGGINGFACE_TOKEN)